# Betweenness Centrality

In this notebook, we will compute the Betweenness centrality for both vertices and edges in our test database using cuGraph and NetworkX. The NetworkX and cuGraph processes will be interleaved so that each step can be compared.

| Author Credit |    Date    |  Update          | cuGraph Version |  Test Hardware |
| --------------|------------|------------------|-----------------|----------------|
| Brad Rees     | 04/24/2019 | created          | 0.15            | GV100, CUDA 11.0
| Brad Rees     | 08/16/2020 | tested / updated | 21.10 nightly   | RTX 3090 CUDA 11.4
| Don Acosta    | 07/05/2022 | tested / updated | 22.08 nightly   | DGX Tesla V100 CUDA 11.5
| Ralph Liu    | 07/26/2022 | updated | 22.08 nightly   | DGX Tesla V100 CUDA 11.5

## Introduction
Betweenness centrality is a measure of the relative importance based on measuring the number of shortest paths that pass through each vertex or over each edge.  High betweenness centrality vertices have a greater number of path cross through the vertex.  Likewise, high centrality edges have more shortest paths that pass over the edge.

See [Betweenness on Wikipedia](https://en.wikipedia.org/wiki/Betweenness_centrality) for more details on the algorithm.



Betweenness centrality of a node 𝑣 is the sum of the fraction of all-pairs shortest paths that pass through 𝑣

<img src="https://latex.codecogs.com/png.latex?c_B(v)&space;=\sum_{s,t&space;\in&space;V}&space;\frac{\sigma(s,&space;t|v)}{\sigma(s,&space;t)}" title="c_B(v) =\sum_{s,t \in V} \frac{\sigma(s, t|v)}{\sigma(s, t)}" />


To compute the Betweenness centrality scores for a graph in cuGraph we use:<br>
__df_v = cugraph.betweenness_centrality(G)__
    G: cugraph.Graph object
   

Returns:

    df: a cudf.DataFrame object with two columns:
        df['vertex']: The vertex identifier for the vertex
        df['betweenness_centrality']: The betweenness centrality score for the vertex



__df_e = cugraph.edge_betweenness_centrality(G)__
    G: cugraph.Graph object

Returns:

    df: a cudf.DataFrame object with two columns:
        df[‘src’]cudf.Series
            Contains the vertex identifiers of the source of each edge
        df[‘dst’]cudf.Series
            Contains the vertex identifiers of the destination of each edge
        df['betweenness_centrality']: The betweenness centrality score for the vertex


### _NOTICE_
cuGraph does not currently support the ‘endpoints’ and ‘weight’ parameters as seen in the corresponding networkX call. 

#### Some notes about vertex IDs...

* cuGraph will automatically renumber graphs to an internal format consisting of a contiguous series of integers starting from 0, and convert back to the original IDs when returning data to the caller. If the vertex IDs of the data are already a contiguous series of integers starting from 0, the auto-renumbering step can be skipped for faster graph creation times.
  * To skip auto-renumbering, set the `renumber` boolean arg to `False` when calling the appropriate graph creation API (eg. `G.from_cudf_edgelist(gdf_r, source='src', destination='dst', renumber=False)`).
  * For more advanced renumbering support, see the examples in `structure/renumber.ipynb` and `structure/renumber-2.ipynb`


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


<img src="../../img/zachary_black_lines.png" width="35%"/>


Because the test data has vertex IDs starting at 1, the auto-renumber feature of cuGraph (mentioned above) will be used so the starting vertex ID is zero for maximum efficiency. The resulting data will then be auto-unrenumbered, making the entire renumbering process transparent to users.


### Prep

In [ ]:
# Import needed libraries
import cugraph
import cudf

# Import a built-in dataset
from cugraph.experimental.datasets import karate

In [ ]:
# NetworkX libraries
import networkx as nx

### Create a Graph 

In [ ]:
# Create a graph using the imported Dataset object
G = karate.get_graph(fetch=True)

### Call the Betweenness Centrality algorithm

In [ ]:
# Call cugraph.betweenness_centrality 
vertex_bc = cugraph.betweenness_centrality(G)

In [ ]:
# Call cugraph.edge_betweenness_centrality 
edge_bc = cugraph.edge_betweenness_centrality(G)

_It was that easy!_  

----

Let's now look at the results

In [ ]:
# let's sort the data and look at the top 5 vertices
vertex_bc.sort_values(by='betweenness_centrality', ascending=False).head(5)

In [ ]:
edge_bc.sort_values(by='betweenness_centrality', ascending=False).head(5)

---

## Now compute using NetworkX

In [ ]:
# Read the data, this also created a NetworkX Graph 
datafile="../../data/karate-data.csv"
file = open(datafile, 'rb')
Gnx = nx.read_edgelist(file)

In [ ]:
bc_nx_vert = nx.betweenness_centrality(Gnx)

In [ ]:
bc_nx_edge = nx.edge_betweenness_centrality(Gnx)

In [ ]:
bc_nx_sv = sorted(((value, key) for (key,value) in bc_nx_vert.items()), reverse=True)
bc_nx_sv[:5]

In [ ]:
bc_nx_se = sorted(((value, key) for (key,value) in bc_nx_edge.items()), reverse=True)
bc_nx_se[:5]

As mentioned, the scores are different but the ranking is the same.

___
Copyright (c) 2019-2022, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___